In [3]:
import numpy as np
from collections import defaultdict
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit.library import PhaseOracle
#from qiskit.test.mock import FakeQuito
from qiskit.compiler import transpile
from qiskit import assemble
from qiskit import IBMQ
from qiskit import execute
from qiskit.tools import job_monitor
from qiskit import *
import matplotlib.pyplot as plt
import mapomatic as mm
IBMQ.save_account('', overwrite=True)
IBMQ.load_account()
#provider = IBMQ.get_provider(hub = 'ibm-q')
provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='vlsi-cad-tools')
device = provider.get_backend('ibmq_kolkata')
#quito = FakeQuito()
circuit_list = []
verification_list = []

In [4]:
class Verifier():
    """Create an object that can be used to check whether
    an assignment satisfies a DIMACS file.
        Args:
            dimacs_file (str): path to the DIMACS file
    """
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
        """Verifies a SAT solution against this object's
        DIMACS file.
            Args:
                guess (str): Assignment to be verified.
                             Must be string of 1s and 0s.
            Returns:
                bool: True if `guess` satisfies the
                           problem. False otherwise.
        """
        # Convert characters to bools & reverse
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    # line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True

In [ ]:
##import pickle
##open_file = open("kol_dimacs/comp_list.pkl", "wb")
##open_verify_file = open("kol_dimacs/veri_list.pkl", "wb")
##pickle.dump(circuit_list, open_file)
##pickle.dump(verification_list, open_verify_file)

##open_file.close()
##open_verify_file.close()

In [ ]:
import pickle
open_file = open("kol_dimacs/comp_list.pkl", "rb")
circuit_list = pickle.load(open_file)
open_file.close()

open_verify_file = open("kol_dimacs/veri_list.pkl", "rb")
verification_list = pickle.load(open_verify_file)
open_file.close()

In [ ]:
#job = device.retrieve_job("6321d827e150f112839ddd0c")
#print(job)

In [ ]:
job = execute(circuit_list,backend = device,shots = 8192)
print(job.job_id())

In [ ]:
job_monitor(job)
device_result = job.result()
#plot_histogram(device_result.get_counts())

In [ ]:
#v = Verifier('dimacs/demosatN3Ands7Ors16NotComp0.7.dimacs')
#v.is_correct('111')
job_counter = 0
N=9
for andIter in range(N*2,N*2+1):#the ands (N*2,N*2+1)
    for orIter in range(andIter*2, andIter*2 + 1): #the ors (andIter*2,(N-1)*(andIter+1)+1, N)
        for notIter in range(3, 9, 2): #not composition
            thenotcomp = notIter*0.1
            filename = "kol_dimacs/demosatN"+str(N)+"Ands"+str(andIter)+"Ors"+str(orIter)+"NotComp"+format(thenotcomp, '.1f')+".dimacs"
            v = Verifier(filename)
            #print(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get))
            print(v.is_correct(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get)))
            job_counter +=1

In [ ]:
print(len(device_result.get_counts()))